### DEEPSEEK UN QWEN MODEĻU ATBILŽU PAPILDUS APSTRĀDEI ASPEKTU IZGŪŠANĀ

In [ ]:
# OpenAI. GPT-4o. Uzvedne: pēc visām izskaidrotajām niansēm uzraksti pilnvērtīgu kodu, kas ļautu šīs atbildes apstrādāt. TP FP FN es ievietošu pašā kodā piem TP=476, FP=256, FN=123 https://chatgpt.com/ [izmantots 2025-05-08]
import re
import json
import xml.etree.ElementTree as ET

TP = 133
FP = 158
FN = 377

log_file = "llm_errors_aspects.log"
xml_file = "LVtestGOLD_1.xml"

def get_gold_aspects(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gold_data = {}
    for sentence in root.iter("sentence"):
        sent_id = sentence.get("id")
        if sentence.find("Opinions") is not None:
            aspects = []
            for op in sentence.find("Opinions").findall("Opinion"):
                target = op.get("target")
                if target and target.lower() != "null":
                    aspects.append(target.lower())
            if aspects:
                gold_data[sent_id] = aspects
    return gold_data

def extract_last_json(text):
    matches = re.findall(r"\{[\s\S]*?\"results\"\s*:\s*\[.*?\]\}", text)
    if matches:
        try:
            return json.loads(matches[-1])
        except:
            return None
    return None

gold_aspects_dict = get_gold_aspects(xml_file)
errors = []

with open(log_file, encoding="utf-8") as f:
    content = f.read()

entries = re.split(r"\n(?=\w+_[^:]+:\d+ — Extra data)", content)

for entry in entries:
    if not entry.strip():
        continue

    match_id = re.match(r"(\w+_\w+_\d+:\d+)", entry)
    if not match_id:
        continue

    sent_id = match_id.group(1)
    gold_aspects = gold_aspects_dict.get(sent_id, [])

    FN -= len(gold_aspects)

    predicted = extract_last_json(entry)
    if predicted is None or "results" not in predicted:
        FN += len(gold_aspects)
        errors.append({
            "id": sent_id,
            "error_type": "NO_JSON",
            "predicted": None,
            "gold": gold_aspects
        })
        continue

    predicted_aspects = [asp.lower() for asp in predicted["results"]]
    gold_set = set(gold_aspects)
    pred_set = set(predicted_aspects)

    # TP: korekti atrastie aspekti
    tp_aspects = gold_set & pred_set
    TP += len(tp_aspects)

    # FP: modeļa liekie aspekti
    fp_aspects = pred_set - gold_set
    FP += len(fp_aspects)
    for asp in fp_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FP",
            "predicted": asp,
            "gold": gold_aspects
        })

    # FN: aspekti, kurus modelis izlaida
    fn_aspects = gold_set - pred_set
    FN += len(fn_aspects)
    for asp in fn_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FN",
            "missed_gold_aspect": asp,
            "predicted": predicted_aspects
        })

precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print("\nPĀRRĒĶINĀTĀS METRIKAS:")
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

with open("adjusted_llm_errors.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=2)
print("\nSaglabātas kļūdas failā: adjusted_llm_errors.json")


### MISTRAL MODEĻA ATBILŽU PAPILDUS APSTRĀDEI ASPEKTU IZGŪŠANĀ

In [ ]:
# OpenAI. GPT-4o. Uzvedne: pēc visām izskaidrotajām niansēm uzraksti pilnvērtīgu kodu, kas ļautu šīs atbildes apstrādāt. TP FP FN es ievietošu pašā kodā piem TP=476, FP=256, FN=123 https://chatgpt.com/ [izmantots 2025-05-08]

import re
import json
import xml.etree.ElementTree as ET

TP = 191
FP = 334
FN = 311

log_file = "llm_errors_aspects.log"
xml_file = "LVtestGOLD_1.xml"

def get_gold_aspects(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gold_data = {}
    for sentence in root.iter("sentence"):
        sent_id = sentence.get("id")
        if sentence.find("Opinions") is not None:
            aspects = []
            for op in sentence.find("Opinions").findall("Opinion"):
                target = op.get("target")
                if target and target.lower() != "null":
                    aspects.append(target.lower())
            if aspects:
                gold_data[sent_id] = aspects
    return gold_data

def extract_first_json(text):
    matches = re.findall(r"\{[\s\S]*?\"results\"\s*:\s*\[.*?\]\}", text)
    if matches:
        try:
            return json.loads(matches[0])
        except:
            return None
    return None

gold_aspects_dict = get_gold_aspects(xml_file)
errors = []

with open(log_file, encoding="utf-8") as f:
    content = f.read()

entries = re.split(r"\n(?=\w+_[^:]+:\d+ — Extra data)", content)

for entry in entries:
    if not entry.strip():
        continue

    match_id = re.match(r"(\w+_\w+_\d+:\d+)", entry)
    if not match_id:
        continue

    sent_id = match_id.group(1)
    gold_aspects = gold_aspects_dict.get(sent_id, [])

    FN -= len(gold_aspects)

    predicted = extract_first_json(entry)
    if predicted is None or "results" not in predicted:
        FN += len(gold_aspects)
        errors.append({
            "id": sent_id,
            "error_type": "NO_JSON",
            "predicted": None,
            "gold": gold_aspects
        })
        continue

    predicted_aspects = [asp.lower() for asp in predicted["results"]]
    gold_set = set(gold_aspects)
    pred_set = set(predicted_aspects)

    # TP: korekti atrastie aspekti
    tp_aspects = gold_set & pred_set
    TP += len(tp_aspects)

    # FP: liekie aspekti
    fp_aspects = pred_set - gold_set
    FP += len(fp_aspects)
    for asp in fp_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FP",
            "predicted": asp,
            "gold": gold_aspects
        })

    # FN: aspekti, ko modelis neizgūst
    fn_aspects = gold_set - pred_set
    FN += len(fn_aspects)
    for asp in fn_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FN",
            "missed_gold_aspect": asp,
            "predicted": predicted_aspects
        })

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\nPĀRRĒĶINĀTĀS METRIKAS (Mistral):")
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

with open("adjusted_llm_errors_mistral.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=2)

print("\nSaglabātas kļūdas failā: adjusted_llm_errors_mistral.json")


### DEEPSEEK UN QWEN MODEĻU ATBILŽU PAPILDUS APSTRĀDEI ASPEKTU NOSKAŅOJUMU NOTEIKŠANĀ

In [ ]:
# OpenAI. GPT-4o. Uzvedne: nepieciešams pārveidot šo kodu tā, lai pēc tāda paša principa tiktu pārbaudīti izgūtie noskaņojumi no modeļu garajām atbildēm https://chatgpt.com/ [izmantots 2025-05-08]

import re
import json
import xml.etree.ElementTree as ET

TP = 476
FP = 65
FN = 173

log_file = "llm_errors_sentiments.log"
xml_file = "testGOLD2016_1.xml"

def get_gold_sentiments(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gold_data = {}
    for sentence in root.iter("sentence"):
        sent_id = sentence.get("id")
        aspects = []
        sentiments = []
        if sentence.find("Opinions") is not None:
            for op in sentence.find("Opinions").findall("Opinion"):
                target = op.get("target")
                polarity = op.get("polarity")
                if target and target.lower() != "null":
                    aspects.append(target.lower())
                    sentiments.append(polarity.lower())
        if aspects:
            gold_data[sent_id] = sentiments
    return gold_data

def extract_last_json(text):
    matches = re.findall(r"\{[\s\S]*?\"results\"\s*:\s*\[.*?\]\}", text)
    if matches:
        try:
            return json.loads(matches[-1])
        except:
            return None
    return None

gold_sentiments_dict = get_gold_sentiments(xml_file)
errors = []

with open(log_file, encoding="utf-8") as f:
    content = f.read()

entries = re.split(r"\n(?=\w+_[^:]+:\d+ — Extra data)", content)

for entry in entries:
    if not entry.strip():
        continue

    match_id = re.match(r"(\w+_\w+_\d+:\d+)", entry)
    if not match_id:
        continue

    sent_id = match_id.group(1)
    gold_sentiments = gold_sentiments_dict.get(sent_id, [])

    FN -= len(gold_sentiments)

    predicted = extract_last_json(entry)
    if predicted is None or "results" not in predicted:
        FN += len(gold_sentiments)
        errors.append({
            "id": sent_id,
            "error_type": "NO_JSON",
            "predicted": None,
            "gold": gold_sentiments
        })
        continue

    predicted_sentiments = predicted["results"]
    valid_labels = {"positive", "negative", "neutral"}
    corrected_preds = [s.lower().strip() for s in predicted_sentiments if isinstance(s, str)]

    for i, true_label in enumerate(gold_sentiments):
        try:
            pred_label = corrected_preds[i]
        except IndexError:
            FN += 1
            errors.append({
                "id": sent_id,
                "error_type": "Missing sentiment",
                "true_sentiment": true_label,
                "predicted_sentiment": None
            })
            continue

        if pred_label not in valid_labels:
            FN += 1
            errors.append({
                "id": sent_id,
                "error_type": "Invalid label",
                "true_sentiment": true_label,
                "predicted_sentiment": pred_label
            })
        elif pred_label == true_label:
            TP += 1
        else:
            FP += 1
            FN += 1
            errors.append({
                "id": sent_id,
                "error_type": "Incorrect sentiment",
                "true_sentiment": true_label,
                "predicted_sentiment": pred_label
            })

    if len(corrected_preds) > len(gold_sentiments):
        for extra in corrected_preds[len(gold_sentiments):]:
            FP += 1
            errors.append({
                "id": sent_id,
                "error_type": "Extra prediction",
                "true_sentiment": None,
                "predicted_sentiment": extra
            })

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\nPĀRRĒĶINĀTĀS SENTIMENTU METRIKAS:")
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

with open("adjusted_llm_sentiment_errors.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=2)

print("\nSaglabātas kļūdas failā: adjusted_llm_sentiment_errors.json")


### DEEPSEEK UN QWEN MODEĻU ATBILŽU PAPILDUS APSTRĀDEI ASPEKTU IZGŪŠANĀ AR LEMMATIZĀCIJU

In [ ]:
! pip install stanza

In [ ]:
# OpenAI. GPT-4o. Uzvedne: pēc visām izskaidrotajām niansēm uzraksti pilnvērtīgu kodu, kas ļautu šīs atbildes apstrādāt. TP FP FN es ievietošu pašā kodā piem TP=476, FP=256, FN=123 https://chatgpt.com/ [izmantots 2025-05-08]
# OpenAI. GPT-4o. Uzvedne: kā ieviest lemmatizāciju tā, lai lemmatizācija aspektiem tiktu veikta pašā pārbaudē, kur tiek skaitīti TP FP FN? https://chatgpt.com/ [izmantots 2025-05-08]

import stanza

stanza.download("lv")
nlp = stanza.Pipeline("lv", processors="tokenize,pos,lemma", tokenize_no_ssplit=True)
def lemmatize_phrase(text):
    doc = nlp(text.lower())
    return " ".join([word.lemma for sent in doc.sentences for word in sent.words])


import re
import json
import xml.etree.ElementTree as ET

TP = 147
FP = 124
FN = 495

log_file = "llm_errors_aspects.log"
xml_file = "testGOLD2016_1.xml"

def get_gold_aspects(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gold_data = {}
    for sentence in root.iter("sentence"):
        sent_id = sentence.get("id")
        if sentence.find("Opinions") is not None:
            aspects = []
            for op in sentence.find("Opinions").findall("Opinion"):
                target = op.get("target")
                if target and target.lower() != "null":
                    aspects.append(target.lower())
            if aspects:
                gold_data[sent_id] = aspects
    return gold_data

def extract_last_json(text):
    matches = re.findall(r"\{[\s\S]*?\"results\"\s*:\s*\[.*?\]\}", text)
    if matches:
        try:
            return json.loads(matches[-1])
        except:
            return None
    return None

gold_aspects_dict = get_gold_aspects(xml_file)
errors = []

with open(log_file, encoding="utf-8") as f:
    content = f.read()

entries = re.split(r"\n(?=\w+_[^:]+:\d+ — Extra data)", content)

for entry in entries:
    if not entry.strip():
        continue

    match_id = re.match(r"(\w+_\w+_\d+:\d+)", entry)
    if not match_id:
        continue

    sent_id = match_id.group(1)
    gold_aspects = gold_aspects_dict.get(sent_id, [])

    FN -= len(gold_aspects)

    predicted = extract_last_json(entry)
    if predicted is None or "results" not in predicted:
        FN += len(gold_aspects)
        errors.append({
            "id": sent_id,
            "error_type": "NO_JSON",
            "predicted": None,
            "gold": gold_aspects
        })
        continue

    # Lemmatizē visus aspektus
    predicted_aspects = [lemmatize_phrase(asp) for asp in predicted["results"]]
    gold_aspects_lemmas = [lemmatize_phrase(asp) for asp in gold_aspects]

    gold_set = set(gold_aspects_lemmas)
    pred_set = set(predicted_aspects)


    # TP: korekti atrastie aspekti
    tp_aspects = gold_set & pred_set
    TP += len(tp_aspects)

    # FP: modeļa liekie aspekti
    fp_aspects = pred_set - gold_set
    FP += len(fp_aspects)
    for asp in fp_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FP",
            "predicted": asp,
            "gold": gold_aspects
        })

    # FN: aspekti, kurus modelis izlaida
    fn_aspects = gold_set - pred_set
    FN += len(fn_aspects)
    for asp in fn_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FN",
            "missed_gold_aspect": asp,
            "predicted": predicted_aspects
        })


precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print("\nPĀRRĒĶINĀTĀS METRIKAS:")
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

with open("adjusted_llm_errors.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=2)
print("\nSaglabātas kļūdas failā: adjusted_llm_errors.json")


### MISTRAL MODEĻA ATBILŽU PAPILDUS APSTRĀDEI ASPEKTU IZGŪŠANĀ AR LEMMATIZĀCIJU

In [ ]:
! pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
# OpenAI. GPT-4o. Uzvedne: pēc visām izskaidrotajām niansēm uzraksti pilnvērtīgu kodu, kas ļautu šīs atbildes apstrādāt. TP FP FN es ievietošu pašā kodā piem TP=476, FP=256, FN=123 https://chatgpt.com/ [izmantots 2025-05-08]
# OpenAI. GPT-4o. Uzvedne: kā ieviest lemmatizāciju tā, lai lemmatizācija aspektiem tiktu veikta pašā pārbaudē, kur tiek skaitīti TP FP FN? https://chatgpt.com/ [izmantots 2025-05-08]

import stanza

stanza.download("lv")
nlp = stanza.Pipeline("lv", processors="tokenize,pos,lemma", tokenize_no_ssplit=True)
def lemmatize_phrase(text):
    doc = nlp(text.lower())
    return " ".join([word.lemma for sent in doc.sentences for word in sent.words])

import re
import json
import xml.etree.ElementTree as ET

TP = 197
FP = 338
FN = 307

log_file = "llm_errors_aspects.log"
xml_file = "testGOLD2016_1.xml"

def get_gold_aspects(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    gold_data = {}
    for sentence in root.iter("sentence"):
        sent_id = sentence.get("id")
        if sentence.find("Opinions") is not None:
            aspects = []
            for op in sentence.find("Opinions").findall("Opinion"):
                target = op.get("target")
                if target and target.lower() != "null":
                    aspects.append(target.lower())
            if aspects:
                gold_data[sent_id] = aspects
    return gold_data

def extract_first_json(text):
    matches = re.findall(r"\{[\s\S]*?\"results\"\s*:\s*\[.*?\]\}", text)
    if matches:
        try:
            return json.loads(matches[0])
        except:
            return None
    return None

gold_aspects_dict = get_gold_aspects(xml_file)
errors = []

with open(log_file, encoding="utf-8") as f:
    content = f.read()

entries = re.split(r"\n(?=\w+_[^:]+:\d+ — Extra data)", content)

for entry in entries:
    if not entry.strip():
        continue

    match_id = re.match(r"(\w+_\w+_\d+:\d+)", entry)
    if not match_id:
        continue

    sent_id = match_id.group(1)
    gold_aspects = gold_aspects_dict.get(sent_id, [])

    FN -= len(gold_aspects)

    predicted = extract_first_json(entry)
    if predicted is None or "results" not in predicted:
        FN += len(gold_aspects)
        errors.append({
            "id": sent_id,
            "error_type": "NO_JSON",
            "predicted": None,
            "gold": gold_aspects
        })
        continue

    predicted_aspects = [lemmatize_phrase(asp) for asp in predicted["results"]]
    gold_aspects_lemmas = [lemmatize_phrase(asp) for asp in gold_aspects]

    gold_set = set(gold_aspects_lemmas)
    pred_set = set(predicted_aspects)


    # TP: korekti atrastie aspekti
    tp_aspects = gold_set & pred_set
    TP += len(tp_aspects)

    # FP: liekie aspekti
    fp_aspects = pred_set - gold_set
    FP += len(fp_aspects)
    for asp in fp_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FP",
            "predicted": asp,
            "gold": gold_aspects
        })

    # FN: aspekti, ko modelis neizgūst
    fn_aspects = gold_set - pred_set
    FN += len(fn_aspects)
    for asp in fn_aspects:
        errors.append({
            "id": sent_id,
            "error_type": "FN",
            "missed_gold_aspect": asp,
            "predicted": predicted_aspects
        })

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\nPĀRRĒĶINĀTĀS METRIKAS (Mistral):")
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

with open("adjusted_llm_errors_mistral.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=2)

print("\nSaglabātas kļūdas failā: adjusted_llm_errors_mistral.json")
